## Project: Maritime Fuels LCA
### Notebook 3: Monte Carlo
#### This notebook is developed by Megan Roux and Massimo Pizzol, and includes adaptions of code provided by Massimo Pizzol, Nils Thonemann and Chris Mutel

This notebook relates to work done for the publication 'Consequential LCA of alternative maritime fuels' which can be found at: doi

Begin by importing all relevant packages (ensure you are in the right environment):

In [1]:
# Note to Meg: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib
from lci_to_bw2 import * # this file has to be in the same folder as the notebook and raw data

Set the project (same as in Notebook 1):

In [2]:
bw.projects.set_current("fuels")

Now check which databases are in the project:
(Should be biopshere3, ecoinvent 3.9 conseq, Fuels_WtT, Fuels_TtW and Fuels_WtW)

In [3]:
bw.databases

Databases dictionary with 27 objects, including:
	Fuels_db_SA1
	Fuels_db_SA1a
	Fuels_db_SA1b
	Fuels_db_SA1c
	Fuels_db_SA1d
	Fuels_db_SA1e
	Fuels_db_SA1f
	Fuels_db_SA2
	Fuels_db_SA2B
	Fuels_db_SA2a
Use `list(this object)` to get the complete list.

In [4]:
ei39db = bw.Database("ecoinvent 3.9 conseq")
biodb = bw.Database("biosphere3")

Setting up the LCIA method:

In [5]:
EF31 = [method for method in bw.methods if "EF v3.1" in str(method) 
        and "no LT" not in str(method)
        and "EN15804" not in str(method)]
# EF31

### Monte Carlo simulations

In [6]:
# Uncertainty type: an interger that tells you what type of uncertainty (see website for list). 2 = lognormal distribution https://docs.brightway.dev/en/latest/content/theory/uncertainty.html
# loc: LN(geometric mean)
# scale: LN(geometric standard deviation)
# negative: is the value negative (TRUE) or not (FALSE)

# https://github.com/PoutineAndRosti/Brightway-Seminar-2017/blob/master/Day%201%20PM/Brightway%20and%20uncertainty.ipynb
# https://stats-arrays.readthedocs.io/en/latest/index.html

# What I did: 
# Calculated the natural log of the median and natural log of the geometric standard deviation and used these for 'loc' and 'scale' values

Choose the LCIA method and impact category (e.g. EF3.1 and climate change impact for this study):

In [6]:
mymethod = EF31[1]

The Monte Carlo simulations are done for the 'well-to-wake' scenario:

In [8]:
Fuels_WtT_3 = ['VLSFO',
                'LNG',
                'NH3_CCS',
                'eNH3',
                'eMeOH_DAC',
                'eMeOH_bio',
                'bioMeOH',
                'DME',
                'PO_fast',
                'PO_slow']

In [9]:
Fuels_WtT_MC = [] # list of functional units
for a in Fuels_WtT_3:
    act = bw.Database('Fuels_db_SA1a').get(a)
    functional_unit = {act: 1} # one unit of each process
    Fuels_WtT_MC.append(functional_unit)

In [10]:
mc = bw.MonteCarloLCA(Fuels_WtT_MC[0], mymethod) # important to initialize the MC simulation
next(mc)

0.05188420668470672

In [11]:
mc.redo_lcia(Fuels_WtT_MC[0]) #just a test
print(mc.score)
mc.redo_lcia(Fuels_WtT_MC[1])
print(mc.score)

0.05188420668470672
0.0536386993933808


In [13]:
# Now the real simulation (takes time)
iterations = 1000
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in Fuels_WtT_MC:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[[0.042916950621680605,
  0.07625872749673933,
  0.1897973107791926,
  0.03523183236723995,
  0.14120517687543954,
  0.03772585411564121,
  0.04286054265942704,
  0.07396385862254083,
  0.061958412686819,
  0.1091730931791215],
 [0.04677699737902276,
  0.06458924070814272,
  0.17486138445292096,
  0.05030735643524564,
  0.14035215778163046,
  0.05277244067294491,
  0.06319015391935284,
  0.10412202370259738,
  0.08212555537237018,
  0.366986793933247],
 [0.04773377452605037,
  0.07950013683783058,
  0.14182215576748733,
  0.0772823261288285,
  0.12191840806054269,
  0.05251618438313973,
  0.04943245271921052,
  0.08395891823147934,
  0.03649460729857588,
  0.046391941902368394],
 [0.05132452656638359,
  0.05702902905098434,
  0.40035323295686137,
  0.051713187904906324,
  0.17155700042263128,
  0.053634921776276996,
  0.06657144949317406,
  0.13315539462461642,
  0.04061116094695322,
  3.1509714799332866],
 [0.04310913663425453,
  0.07515783693496977,
  0.13421238520634507,
  0.0461682

In [14]:
df = pd.DataFrame(simulations, columns = Fuels_WtT_3)
df.head()

,VLSFO,LNG,NH3_CCS,eNH3,eMeOH_DAC,eMeOH_bio,bioMeOH,DME,PO_fast,PO_slow
0,0.042917,0.076259,0.189797,0.035232,0.141205,0.037726,0.042861,0.073964,0.061958,0.109173
1,0.046777,0.064589,0.174861,0.050307,0.140352,0.052772,0.063190,0.104122,0.082126,0.366987
2,0.047734,0.079500,0.141822,0.077282,0.121918,0.052516,0.049432,0.083959,0.036495,0.046392
3,0.051325,0.057029,0.400353,0.051713,0.171557,0.053635,0.066571,0.133155,0.040611,3.150971
4,0.043109,0.075158,0.134212,0.046168,0.143683,0.044538,0.052767,0.085044,0.017503,-0.078527


In [15]:
df.to_excel('Results_Fuels_SA1a_MC_1000.xlsx')

Of course, MC calculations can also be run for the 'well-to-tank' and 'tank-to-wake' scenarios:

In [ ]:
Fuels_WtT_2 = ['eMeOH_DAC',
                'eMeOH_bio',
                'bioMeOH',
                'NH3_CCS',
                'eNH3',
                'DME',
                'PO_fast',
                'PO_slow',
                'LNG' , 
                'VLSFO']

Fuels_TtW_2 = ['MeOH_combustion',
                'NH3_combustion',
                'DME_combustion',
                'LNG_combustion',
                'PO_combustion',
                'VLSFO_combustion']

In [ ]:
fus = [] # list of functional units
for a in Fuels_WtT_2:
    act = bw.Database('Fuels_db_WtT_MJ').get(a)
    functional_unit = {act: 1} # one unit of each process
    fus.append(functional_unit)

In [ ]:
mc = bw.MonteCarloLCA(fus[0], mymethod) # important to initialize the MC simulation
next(mc)

In [ ]:
mc.redo_lcia(fus[0]) #just a test
print(mc.score)
mc.redo_lcia(fus[1])
print(mc.score)

In [ ]:
iterations = 100 
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in fus:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

In [ ]:
df = pd.DataFrame(simulations, columns = Fuels_WtT_2)
df.head()

In [ ]:
df.to_excel('Results_Fuels_WtT_MJ_EF31_MC.xlsx')

In [ ]:
Fuels_TtW_MC = [] # list of functional units
for a in Fuels_TtW_2:
    act = bw.Database('Fuels_db_TtW_MJ').get(a)
    functional_unit = {act: 1} # one unit of each process
    Fuels_TtW_MC.append(functional_unit)

In [ ]:
mc = bw.MonteCarloLCA(Fuels_TtW_MC[0], mymethod) # important to initialize the MC simulation
next(mc)

In [ ]:
mc.redo_lcia(Fuels_TtW_MC[0]) #just a test
print(mc.score)
mc.redo_lcia(Fuels_TtW_MC[1])
print(mc.score)

In [ ]:
iterations = 100
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in Fuels_TtW_MC:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

In [ ]:
df = pd.DataFrame(simulations, columns = Fuels_TtW_2)
df.head()

In [ ]:
df.to_excel('Results_Fuels_TtW_MJ_EF31_MC.xlsx')